In [1]:
from bs4 import BeautifulSoup
import html5lib
from bs4 import BeautifulSoup

In [42]:
url = 'https://gnomad.broadinstitute.org/api/'



In [44]:
header2 = {
    "Host": "gnomad.broadinstitute.org",
"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0",
"Accept": "*/*",
"Accept-Language": "en-US,en;q=0.5",
"Accept-Encoding": "gzip, deflate, br",
"Referer": "https://gnomad.broadinstitute.org/region/4-1002741-1002771?dataset=gnomad_r3",
"content-type": "application/json",
"Origin": "https://gnomad.broadinstitute.org",
"Content-Length": "992",
"Connection": "keep-alive",
"Cookie": "_ga=GA1.2.1956078262.1593106327; _gid=GA1.2.2001994588.1593811914; _gat_gtag_UA_85987017_1=1",
"TE": "Trailers"
}

In [ ]:
{"Request Headers (500 B)":{"headers":[{"name":"Accept","value":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"},{"name":"Accept-Encoding","value":"gzip, deflate, br"},{"name":"Accept-Language","value":"en-US,en;q=0.5"},{"name":"Cache-Control","value":"no-cache"},{"name":"Connection","value":"keep-alive"},{"name":"Cookie","value":"_ga=GA1.2.1956078262.1593106327; _gid=GA1.2.2001994588.1593811914"},{"name":"Host","value":"gnomad.broadinstitute.org"},{"name":"Pragma","value":"no-cache"},{"name":"Upgrade-Insecure-Requests","value":"1"},{"name":"User-Agent","value":"Mozilla/5.0 (X11; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0"}]}}

In [45]:
r = requests.post(url, headers=header2)
r.status_code

KeyboardInterrupt: 

In [46]:
soup = BeautifulSoup(r.content, 'html.parser')

In [47]:
print(soup.prettify())

<html>
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <title>
   408 Request Timeout
  </title>
 </head>
 <body bgcolor="#ffffff" text="#000000">
  <h1>
   Error: Request Timeout
  </h1>
  <h2>
   Your client has taken too long to issue its request.
  </h2>
  <h2>
  </h2>
 </body>
</html>



In [48]:
import urllib.parse
urllib.parse.unquote('https://gnomad.broadinstitute.org/api/?query=query%20RegionCoverage(%24chrom%3A%20String!%2C%20%24start%3A%20Int!%2C%20%24stop%3A%20Int!%2C%20%24datasetId%3A%20DatasetId!%2C%20%24referenceGenome%3A%20ReferenceGenomeId!%2C%20%24includeExomeCoverage%3A%20Boolean!%2C%20%24includeGenomeCoverage%3A%20Boolean!)%20%7B%0A%20%20region(chrom%3A%20%24chrom%2C%20start%3A%20%24start%2C%20stop%3A%20%24stop%2C%20reference_genome%3A%20%24referenceGenome)%20%7B%0A%20%20%20%20exome_coverage(dataset%3A%20%24datasetId)%20%40include(if%3A%20%24includeExomeCoverage)%20%7B%0A%20%20%20%20%20%20pos%0A%20%20%20%20%20%20mean%0A%20%20%20%20%20%20median%0A%20%20%20%20%20%20over_1%0A%20%20%20%20%20%20over_5%0A%20%20%20%20%20%20over_10%0A%20%20%20%20%20%20over_15%0A%20%20%20%20%20%20over_20%0A%20%20%20%20%20%20over_25%0A%20%20%20%20%20%20over_30%0A%20%20%20%20%20%20over_50%0A%20%20%20%20%20%20over_100%0A%20%20%20%20%7D%0A%20%20%20%20genome_coverage(dataset%3A%20%24datasetId)%20%40include(if%3A%20%24includeGenomeCoverage)%20%7B%0A%20%20%20%20%20%20pos%0A%20%20%20%20%20%20mean%0A%20%20%20%20%20%20median%0A%20%20%20%20%20%20over_1%0A%20%20%20%20%20%20over_5%0A%20%20%20%20%20%20over_10%0A%20%20%20%20%20%20over_15%0A%20%20%20%20%20%20over_20%0A%20%20%20%20%20%20over_25%0A%20%20%20%20%20%20over_30%0A%20%20%20%20%20%20over_50%0A%20%20%20%20%20%20over_100%0A%20%20%20%20%7D%0A%20%20%7D%0A%7D%0A')

'https://gnomad.broadinstitute.org/api/?query=query RegionCoverage($chrom: String!, $start: Int!, $stop: Int!, $datasetId: DatasetId!, $referenceGenome: ReferenceGenomeId!, $includeExomeCoverage: Boolean!, $includeGenomeCoverage: Boolean!) {\n  region(chrom: $chrom, start: $start, stop: $stop, reference_genome: $referenceGenome) {\n    exome_coverage(dataset: $datasetId) @include(if: $includeExomeCoverage) {\n      pos\n      mean\n      median\n      over_1\n      over_5\n      over_10\n      over_15\n      over_20\n      over_25\n      over_30\n      over_50\n      over_100\n    }\n    genome_coverage(dataset: $datasetId) @include(if: $includeGenomeCoverage) {\n      pos\n      mean\n      median\n      over_1\n      over_5\n      over_10\n      over_15\n      over_20\n      over_25\n      over_30\n      over_50\n      over_100\n    }\n  }\n}\n'

In [1]:
from pandas import json_normalize as json_normalize
from tqdm import tqdm
import pandas as pd
import requests
import argparse
import json
import os

In [26]:
query = """
    {
      %s(%s: "%s") {
        variants(dataset: %s) {
          gene_id
          gene_symbol
          chrom
          pos
          rsid
          ref
          alt
          consequence
          genome {
            genome_af:af
            genome_ac:ac
            genome_an:an
            genome_ac_hemi:ac_hemi
            genome_ac_hom:ac_hom
          }
          exome {
            exome_af:af
            exome_ac:ac
            exome_an:an
            exome_ac_hemi:ac_hemi
            exome_ac_hom:ac_hom
          }
          flags
          lof
          consequence_in_canonical_transcript
          gene_symbol
          hgvsc
          lof_filter
          lof_flags
          hgvsc
          hgvsp
          reference_genome
          variant_id: variantId
        }
      }
    }
    """

query = query % ("gene", "gene_name", "TP53", "gnomad_r2_1")
query

'\n    {\n      gene(gene_name: "TP53") {\n        variants(dataset: gnomad_r2_1) {\n          gene_id\n          gene_symbol\n          chrom\n          pos\n          rsid\n          ref\n          alt\n          consequence\n          genome {\n            genome_af:af\n            genome_ac:ac\n            genome_an:an\n            genome_ac_hemi:ac_hemi\n            genome_ac_hom:ac_hom\n          }\n          exome {\n            exome_af:af\n            exome_ac:ac\n            exome_an:an\n            exome_ac_hemi:ac_hemi\n            exome_ac_hom:ac_hom\n          }\n          flags\n          lof\n          consequence_in_canonical_transcript\n          gene_symbol\n          hgvsc\n          lof_filter\n          lof_flags\n          hgvsc\n          hgvsp\n          reference_genome\n          variant_id: variantId\n        }\n      }\n    }\n    '

In [29]:
query = "\n  query FetchRegion($chrom: String!, $start: Int!, $stop: Int!, $referenceGenome: ReferenceGenomeId!) {\n    region(chrom: $chrom, start: $start, stop: $stop, reference_genome: $referenceGenome) {\n      reference_genome\n      chrom\n      start\n      stop\n      genes {\n        gene_id\n        symbol\n        start\n        stop\n        exons {\n          feature_type\n          start\n          stop\n        }\n      }\n    }\n  }\n"
query

'\n  query FetchRegion($chrom: String!, $start: Int!, $stop: Int!, $referenceGenome: ReferenceGenomeId!) {\n    region(chrom: $chrom, start: $start, stop: $stop, reference_genome: $referenceGenome) {\n      reference_genome\n      chrom\n      start\n      stop\n      genes {\n        gene_id\n        symbol\n        start\n        stop\n        exons {\n          feature_type\n          start\n          stop\n        }\n      }\n    }\n  }\n'

In [54]:
variables = """{"chrom":"4","start":1002741,"stop":1002771,"referenceGenome":"GRCh38","datasetId":"gnomad_r3"}"""
variables

'{"chrom":"4","start":1002741,"stop":1002771,"referenceGenome":"GRCh38","datasetId":"gnomad_r3"}'

In [45]:
end_point = "https://gnomad.broadinstitute.org/api/"
response = requests.post(end_point, data={'query': query, 'variables': variables}, timeout=None)
#data = json_normalize(response.json()["data"]["region"]["reference_genome"])
#data.columns = data.columns.map(lambda x: x.split(".")[-1])
data = json_normalize(response.json()["data"]["region"]["genes"])
data

,gene_id,symbol,start,stop,exons
0,ENSG00000127415,IDUA,986997,1004506,"[{'feature_type': 'CDS', 'start': 987085, 'sto..."


In [32]:
response.json()

{'data': {'region': {'reference_genome': 'GRCh38',
   'chrom': '4',
   'start': 1002741,
   'stop': 1002771,
   'genes': [{'gene_id': 'ENSG00000127415',
     'symbol': 'IDUA',
     'start': 986997,
     'stop': 1004506,
     'exons': [{'feature_type': 'CDS', 'start': 987085, 'stop': 987242},
      {'feature_type': 'CDS', 'start': 987809, 'stop': 987949},
      {'feature_type': 'CDS', 'start': 991825, 'stop': 991940},
      {'feature_type': 'CDS', 'start': 992139, 'stop': 992230},
      {'feature_type': 'CDS', 'start': 1000612, 'stop': 1000697},
      {'feature_type': 'CDS', 'start': 1000882, 'stop': 1000989},
      {'feature_type': 'CDS', 'start': 1001468, 'stop': 1001563},
      {'feature_type': 'CDS', 'start': 1001679, 'stop': 1001881},
      {'feature_type': 'CDS', 'start': 1001982, 'stop': 1002161},
      {'feature_type': 'CDS', 'start': 1002269, 'stop': 1002485},
      {'feature_type': 'CDS', 'start': 1002732, 'stop': 1002944},
      {'feature_type': 'CDS', 'start': 1003036, 'stop

In [72]:
query = """query VariantInRegion($chrom: String!, $start: Int!, $stop: Int!, $datasetId: DatasetId!, $referenceGenome: ReferenceGenomeId!) {
  region(start: $start, stop: $stop, chrom: $chrom, reference_genome: $referenceGenome) {
    clinvar_variants {
      clinical_significance
      clinvar_variation_id
      gold_stars
      major_consequence
      pos
      variant_id
    }
    variants(dataset: $datasetId) {
      consequence
      flags
      gene_id
      gene_symbol
      hgvs
      hgvsc
      hgvsp
      lof
      lof_filter
      lof_flags
      pos
      rsid
      variant_id: variantId
      exome {
        ac
        ac_hemi
        ac_hom
        an
        af
        filters
        populations {
          id
          ac
          an
          ac_hemi
          ac_hom
        }
      }
      genome {
        ac
        ac_hemi
        ac_hom
        an
        af
        filters
        populations {
          id
          ac
          an
          ac_hemi
          ac_hom
        }
      }
    }
  }
}"""
query

'query VariantInRegion($chrom: String!, $start: Int!, $stop: Int!, $datasetId: DatasetId!, $referenceGenome: ReferenceGenomeId!) {\n  region(start: $start, stop: $stop, chrom: $chrom, reference_genome: $referenceGenome) {\n    clinvar_variants {\n      clinical_significance\n      clinvar_variation_id\n      gold_stars\n      major_consequence\n      pos\n      variant_id\n    }\n    variants(dataset: $datasetId) {\n      consequence\n      flags\n      gene_id\n      gene_symbol\n      hgvs\n      hgvsc\n      hgvsp\n      lof\n      lof_filter\n      lof_flags\n      pos\n      rsid\n      variant_id: variantId\n      exome {\n        ac\n        ac_hemi\n        ac_hom\n        an\n        af\n        filters\n        populations {\n          id\n          ac\n          an\n          ac_hemi\n          ac_hom\n        }\n      }\n      genome {\n        ac\n        ac_hemi\n        ac_hom\n        an\n        af\n        filters\n        populations {\n          id\n          ac\n  

In [75]:
variables = """
    {
    "chrom":"4",
    "datasetId":"gnomad_r3",
    "referenceGenome":"GRCh38",
    "start":1002741,
    "stop":1002771}
"""
variables

'\n    {\n    "chrom":"4",\n    "datasetId":"gnomad_r3",\n    "referenceGenome":"GRCh38",\n    "start":1002741,\n    "stop":1002771}\n'

In [76]:
end_point = "https://gnomad.broadinstitute.org/api/"
response = requests.post(end_point, data={'query': query, 'variables': variables}, timeout=None)
response.json()

{'data': {'region': {'clinvar_variants': [{'clinical_significance': ['Pathogenic'],
     'clinvar_variation_id': '11908',
     'gold_stars': 2,
     'major_consequence': 'stop_gained',
     'pos': 1002747,
     'variant_id': '4-1002747-G-A'},
    {'clinical_significance': ['Pathogenic'],
     'clinvar_variation_id': '555230',
     'gold_stars': 1,
     'major_consequence': 'stop_gained',
     'pos': 1002748,
     'variant_id': '4-1002748-G-A'},
    {'clinical_significance': ['Pathogenic'],
     'clinvar_variation_id': '552095',
     'gold_stars': 2,
     'major_consequence': 'stop_gained',
     'pos': 1002752,
     'variant_id': '4-1002752-G-T'},
    {'clinical_significance': ['Likely_benign'],
     'clinvar_variation_id': '724507',
     'gold_stars': 1,
     'major_consequence': 'synonymous_variant',
     'pos': 1002766,
     'variant_id': '4-1002766-C-T'},
    {'clinical_significance': ['Conflicting_interpretations_of_pathogenicity'],
     'clinvar_variation_id': '92627',
     'gold_

In [95]:
inRegionQuery = """query VariantInRegion($chrom: String!, $start: Int!, $stop: Int!, $datasetId: DatasetId!, $referenceGenome: ReferenceGenomeId!) {
  region(start: $start, stop: $stop, chrom: $chrom, reference_genome: $referenceGenome) {
    clinvar_variants {
      clinical_significance
      clinvar_variation_id
      gold_stars
      major_consequence
      pos
      variant_id
    }
    variants(dataset: $datasetId) {
      consequence
      flags
      gene_id
      gene_symbol
      hgvs
      hgvsc
      hgvsp
      lof
      lof_filter
      lof_flags
      pos
      rsid
      variant_id: variantId
      exome {
        ac
        ac_hemi
        ac_hom
        an
        af
        filters
        populations {
          id
          ac
          an
          ac_hemi
          ac_hom
        }
      }
      genome {
        ac
        ac_hemi
        ac_hom
        an
        af
        filters
        populations {
          id
          ac
          an
          ac_hemi
          ac_hom
        }
      }
    }
  }
}"""
    
regionCoverageQuery = """query RegionCoverage($chrom: String!, $start: Int!, $stop: Int!, $datasetId: DatasetId!, $referenceGenome: ReferenceGenomeId!, $includeExomeCoverage: Boolean!, $includeGenomeCoverage: Boolean!) {
  region(chrom: $chrom, start: $start, stop: $stop, reference_genome: $referenceGenome) {
    exome_coverage(dataset: $datasetId) @include(if: $includeExomeCoverage) {
      pos
      mean
      median
      over_1
      over_5
      over_10
      over_15
      over_20
      over_25
      over_30
      over_50
      over_100
    }
    genome_coverage(dataset: $datasetId) @include(if: $includeGenomeCoverage) {
      pos
      mean
      median
      over_1
      over_5
      over_10
      over_15
      over_20
      over_25
      over_30
      over_50
      over_100
    }
  }
}"""
    
fetchRegionQuery = """query FetchRegion($chrom: String!, $start: Int!, $stop: Int!, $referenceGenome: ReferenceGenomeId!) {
    region(chrom: $chrom, start: $start, stop: $stop, reference_genome: $referenceGenome) {
      reference_genome
      chrom
      start
      stop
      genes {
        gene_id
        symbol
        start
        stop
        exons {
          feature_type
          start
          stop
        }
      }
    }
  }"""

class VariantsSearch:
    
    def __init__(self, chromossome, dataset, start, end):
        self.inRegion = inRegionQuery
        self.regionCoverage = regionCoverageQuery
        self.fetchRegion = fetchRegionQuery
        
        self.chrom = chromossome
        self.dataset = dataset
        self.start = start
        self.end = end
        
    
    @classmethod
    def requestGnomad(self, query, variables):
        end_point = "https://gnomad.broadinstitute.org/api/"
        response = requests.post(end_point, data={'query': query, 'variables': variables}, timeout=None)
        return response.json()
    
    
    def getInRegion(self):
        
        variables = """
            {
                "chrom":"%s",
                "datasetId":"%s",
                "referenceGenome":"GRCh38",
                "start":%s,
                "stop":%s
            }""" % (self.chrom, self.dataset, self.start, self.end)
        
        answer = self.requestGnomad(self.inRegion, variables)
        return answer
    
    
    def getRegionCoverage(self):
        
        variables = """
            {
                "chrom":"%s",
                "datasetId":"%s",
                "includeExomeCoverage":false,
                "includeGenomeCoverage":true,
                "referenceGenome":"GRCh38",
                "start":%s,
                "stop":%s
            }""" % (self.chrom, self.dataset, self.start, self.end)
        
        answer = self.requestGnomad(self.regionCoverage, variables)
        return answer
    
    
    def getFetchRegion(self):
        
        variables = """
            {
                "chrom":"%s",
                "datasetId":"%s",
                "referenceGenome":"GRCh38",
                "start":%s,
                "stop":%s
            }""" % (self.chrom, self.dataset, self.start, self.end)
        
        answer = self.requestGnomad(self.fetchRegion, variables)
        return answer

In [99]:
vs = VariantsSearch("4", "gnomad_r3", "1002741", "1002771")
vs.getInRegion()
#vs.getRegionCoverage()
#vs.getFetchRegion()

{'data': {'region': {'clinvar_variants': [{'clinical_significance': ['Pathogenic'],
     'clinvar_variation_id': '11908',
     'gold_stars': 2,
     'major_consequence': 'stop_gained',
     'pos': 1002747,
     'variant_id': '4-1002747-G-A'},
    {'clinical_significance': ['Pathogenic'],
     'clinvar_variation_id': '555230',
     'gold_stars': 1,
     'major_consequence': 'stop_gained',
     'pos': 1002748,
     'variant_id': '4-1002748-G-A'},
    {'clinical_significance': ['Pathogenic'],
     'clinvar_variation_id': '552095',
     'gold_stars': 2,
     'major_consequence': 'stop_gained',
     'pos': 1002752,
     'variant_id': '4-1002752-G-T'},
    {'clinical_significance': ['Likely_benign'],
     'clinvar_variation_id': '724507',
     'gold_stars': 1,
     'major_consequence': 'synonymous_variant',
     'pos': 1002766,
     'variant_id': '4-1002766-C-T'},
    {'clinical_significance': ['Conflicting_interpretations_of_pathogenicity'],
     'clinvar_variation_id': '92627',
     'gold_

In [ ]:
#gene posição variante frequência(allele frequency)